In [28]:
from cluster import cluster as cl
rc = cl.start_cluster('homard', 15)

In [21]:
dv = rc[:]

NoEnginesRegistered: Can't build targets without any engines

In [4]:
%%px

Nsystems=33
maxorbs = 1.e6
Nfeatures = 25

import numpy as np
mp = 5*3.e-6 # 5 earth masses
gamma = (2./3.*mp)**(1./3.) # Hill radius with factor of a taken out
a1 = 0.05 # semimajor axis of innermost planet in AU
imax = 1.*np.pi/180. # maximum inclination (uniform distribution)
emax = 1.e-2 # maximum eccentricity (uniform distribution)
betamin = 4.#5.
betamax = 7.#10.

In [5]:
%%px

import rebound
import random
import os
import time

def system():
    random.seed(os.getpid()+time.time())
    
    beta = random.uniform(betamin, betamax)
    a2 = a1 + beta*gamma*a1
    a3 = a2 + beta*gamma*a2
    
    sim = rebound.Simulation()
    sim.G = 4*np.pi**2
    sim.exit_min_distance = a1*gamma # use smaller hill radius as exit condition
    
    
    sim.add(m=1.)
    sim.add(m=mp, a=a1, e=random.random()*emax, pomega=random.random()*2.*np.pi, inc=random.random()*imax, Omega=random.random()*2.*np.pi, f=random.random()*2.*np.pi)
    sim.add(m=mp, a=a2, e=random.random()*emax, pomega=random.random()*2.*np.pi, inc=random.random()*imax, Omega=random.random()*2.*np.pi, f=random.random()*2.*np.pi)
    sim.add(m=mp, a=a3, e=random.random()*emax, pomega=random.random()*2.*np.pi, inc=random.random()*imax, Omega=random.random()*2.*np.pi, f=random.random()*2.*np.pi)
    sim.move_to_com()
    ps = sim.particles
    
    tmax = maxorbs * ps[1].P # number of inner planet orbital periods to integrate
    
    stable = True # assume it's stable and update if not
    try:
        sim.integrate(tmax)
    except rebound.Encounter as error:
        stable = False

    features = [stable, maxorbs, mp, gamma]
    for p in ps[1:]:
        features = features + [p.a, p.P, p.e, p.pomega, p.inc, p.Omega, p.f]
        
    return features

In [ ]:
%%px

import pandas as pd

columns = ('Stable', 'Norbits', 'Mplanet', 'RH/a')
types = ('bool', 'int32', 'float32', 'float32')

for i in ['1', '2', '3']:
    columns += ('a'+i, 'P'+i, 'e'+i, 'pomega'+i, 'inc'+i, 'Omega'+i, 'f'+i)
    types += tuple(['float32']*7)
    
vals = np.zeros(Nsystems, dtype=zip(columns, types))
df = pd.DataFrame(vals)

t0 = time.time()

for i in range(Nsystems):
    df.loc[i] = system()

dt = time.time() - t0

In [9]:
import pandas as pd
dfs = dv['df']
dts = dv['dt']
df = pd.concat(dfs, ignore_index=True)
df.tail()

,Stable,Norbits,Mplanet,RH/a,a1,P1,e1,pomega1,inc1,Omega1,...,inc2,Omega2,f2,a3,P3,e3,pomega3,inc3,Omega3,f3
1000,False,1000000.0,0.000015,0.021544,0.051029,0.011527,0.036488,-0.785860,0.004167,0.909065,...,0.012544,-1.402713,5.571021,0.062228,0.015523,0.017087,1.921845,0.010081,2.195238,-2.620730
1001,True,1000000.0,0.000015,0.021544,0.050007,0.011183,0.005126,1.329891,0.007770,-0.986941,...,0.005684,0.983505,-3.774172,0.072321,0.019449,0.007358,-2.349656,0.005706,-0.817724,-1.323172
1002,False,1000000.0,0.000015,0.021544,0.047912,0.010487,0.091516,-2.727164,0.005107,-1.391558,...,0.001089,-2.684870,-0.729186,0.063547,0.016019,0.059333,0.252285,0.000591,2.633015,1.425648
1003,True,1000000.0,0.000015,0.021544,0.050023,0.011188,0.004134,-3.717838,0.008807,-1.306570,...,0.014346,2.752379,0.613621,0.064639,0.016434,0.005525,2.230810,0.005706,0.570917,-1.691514
1004,True,1000000.0,0.000015,0.021544,0.049999,0.011180,0.010164,-0.524282,0.002483,1.701424,...,0.005298,-0.566092,3.164369,0.071693,0.019196,0.003903,1.597361,0.006829,3.111856,-0.928524


In [17]:
df.to_csv('1e6dataset.txt', encoding='ascii')

In [11]:
print("Times of execution for each engine (s) = {0}".format(dts))

Times of execution for each engine (s) = [44374.88495397568, 49278.750933885574, 44298.07342290878, 51160.483650922775, 42486.595427036285, 43320.219094991684, 50500.70640015602, 47145.03948378563, 46972.21672987938, 45972.759606838226, 45849.23973107338, 48379.56002497673, 47034.66717886925, 45261.85488986969, 41155.19060397148]


In [16]:
print("Maximum (avg) integration time per system in hours = {0}".format(max(dts)/3600/67.))

Maximum (avg) integration time per system in hours = 0.212108141173


In [18]:
cl.shutdown_cluster('homard', rc)